# How PySpark is better than Pandas for data frame operations

## Install PySpark

Before we start this discussion I think it's important to show how easy it is to install Pyspark.  I recommend using Anaconda to do the PySpark installation.  It only involves a few simple steps -- assuming you already have Anaconda working.  I won't go into that here. I noticed that Pandas was one of the dependencies for the PySpark installation, so we kill two birds with one stone.

Open a Terminal and execute the following commands one at a time.

```{bash}
# Step 1: Activate your existing conda env
conda activate pyspark_env

# Step 2: Install Open JDK
conda install openjdk

# Step 3: Install Pyspark
conda install pyspark

# Step 4: Install findspark in order to run Pyspark in a Jupyter notebook
conda install -c conda-forge findspark
```

## Import Python packages

In [15]:
import pandas as pd
import pyspark.sql.types as T
import pyspark.sql.functions as F
import pyspark.sql.window as W
from pyspark.sql import SparkSession

In [2]:
# Create a new spark session
spark = SparkSession.builder.getOrCreate()
# spark = SparkSession.builder.appName("intro").getOrCreate()

22/11/05 19:56:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
# Check to see if our spark session is working
spark.sql("select 'hello world' as greeting").show()

+-----------+
|   greeting|
+-----------+
|hello world|
+-----------+



## Create some data frames in both Pandas and Pyspark

In [26]:
# Field values
player_ids  = [1,2,3,4,5,6,7,8,9,10]
first_names = ['Juan','Manny','Yu','He-Seong','Jurickson','Jake','Brandon','Josh','Trent','Austin']
last_names  = ['Soto', 'Machado', 'Darvish', 'Kim','Profar','Cronenworth','Drury','Bell','Grisham','Nola']
positions   = ['RF','3B','P','SS','LF','2B','1B','DH','CF','C']
batting     = ['L','R',None,'R','S','L','R','S','L','R']
at_bats     = [524,578,None,517,575,587,518,552,451,347]
obp         = [0.401,0.366,None,0.325,0.331,0.332,0.320,0.362,0.284,0.321]
teams       = ['Padres' for x in range(10)]

### Pandas Dataframe

In [40]:
# Create a new Pandas dataframe from the field lists from above
df_pandas = pd.DataFrame({'player_id':player_ids,
                          'team':team,
                          'first_name':first_names,
                          'last_name':last_names,
                          'position':positions,
                          'batting_stance':batting,
                          'at_bats':at_bats,
                          'obp':obp})

In [41]:
df_pandas

,player_id,team,first_name,last_name,position,batting_stance,at_bats,obp
0,1,Padres,Juan,Soto,RF,L,524.0,0.401
1,2,Padres,Manny,Machado,3B,R,578.0,0.366
2,3,Padres,Yu,Darvish,P,None,NaN,NaN
3,4,Padres,He-Seong,Kim,SS,R,517.0,0.325
4,5,Padres,Jurickson,Profar,LF,S,575.0,0.331
5,6,Padres,Jake,Cronenworth,2B,L,587.0,0.332
6,7,Padres,Brandon,Drury,1B,R,518.0,0.320
7,8,Padres,Josh,Bell,DH,S,552.0,0.362
8,9,Padres,Trent,Grisham,CF,L,451.0,0.284
9,10,Padres,Austin,Nola,C,R,347.0,0.321


### Pyspark Dataframe

In [42]:
# Create a new dataframe from the above Pandas dataframe
df_pyspark = spark.createDataFrame(df_pandas)

In [43]:
# We need to covert it back to Pandas to display the dataframe
df_pyspark.toPandas()

,player_id,team,first_name,last_name,position,batting_stance,at_bats,obp
0,1,Padres,Juan,Soto,RF,L,524.0,0.401
1,2,Padres,Manny,Machado,3B,R,578.0,0.366
2,3,Padres,Yu,Darvish,P,None,NaN,NaN
3,4,Padres,He-Seong,Kim,SS,R,517.0,0.325
4,5,Padres,Jurickson,Profar,LF,S,575.0,0.331
5,6,Padres,Jake,Cronenworth,2B,L,587.0,0.332
6,7,Padres,Brandon,Drury,1B,R,518.0,0.320
7,8,Padres,Josh,Bell,DH,S,552.0,0.362
8,9,Padres,Trent,Grisham,CF,L,451.0,0.284
9,10,Padres,Austin,Nola,C,R,347.0,0.321


## Tasks

### Task 1: Combine first and last name fields into a new column called, 'full_name'

#### Pandas solution 1 - Using the `apply` function

This is the most common way I that I have seen people solving this task.  You need to look at it for a bit to understand what is going on.  I'm not a big fan of the apply function.  I think that list comprehensions are a more pythonic way of doing the same task.

In [44]:
df_pandas['full_name'] = df_pandas[["first_name", "last_name"]].apply(" ".join, axis=1)
df_pandas.head(3)

,player_id,team,first_name,last_name,position,batting_stance,at_bats,obp,full_name
0,1,Padres,Juan,Soto,RF,L,524.0,0.401,Juan Soto
1,2,Padres,Manny,Machado,3B,R,578.0,0.366,Manny Machado
2,3,Padres,Yu,Darvish,P,None,NaN,NaN,Yu Darvish


#### Pandas solution 2 - Using a list comprehension and an f-string

Here is a solution that uses the same technique as above, but with a list comprehension instead of an apply function. It may be a bit more "pythonic", but it is still not pretty.

In [45]:
df_pandas['full_name'] = [f"{x[0]} {x[1]}" for x in zip(df_pandas["first_name"], df_pandas["last_name"])]
df_pandas.head(3)

,player_id,team,first_name,last_name,position,batting_stance,at_bats,obp,full_name
0,1,Padres,Juan,Soto,RF,L,524.0,0.401,Juan Soto
1,2,Padres,Manny,Machado,3B,R,578.0,0.366,Manny Machado
2,3,Padres,Yu,Darvish,P,None,NaN,NaN,Yu Darvish


#### Pandas solution 3 - Using the `assign` function

This is my favorite Pandas solution.  I like being able to change dataframe operations in series. I can use the `assign` function to accomplish this task.  Unfortunately, like the previous 2 solutions, it is also bit ugly. It involves using a lambda function, a list comprehension, and the zip iterator function.  It's kind of crazy, but it all fits onto a single line.  Once you memorize the pattern you can do this sort of assignment operation one line after another.

In [46]:
(
    df_pandas
    .assign(full_name = lambda x: [f"{y[0]} {y[1]}" for y in zip(x.first_name, x.last_name)])
    .head(3)
)

,player_id,team,first_name,last_name,position,batting_stance,at_bats,obp,full_name
0,1,Padres,Juan,Soto,RF,L,524.0,0.401,Juan Soto
1,2,Padres,Manny,Machado,3B,R,578.0,0.366,Manny Machado
2,3,Padres,Yu,Darvish,P,None,NaN,NaN,Yu Darvish


#### Pyspark solution

I have saved the best solution for last.  As you saw above, there is no straight forward way to accomplish this task using Pandas.  This Pyspark solution is just as wordy.  However, in my opinion, it way more straight forward.  It is just a few funtion calls.

In [47]:
(
    df_pyspark
    .withColumn('full_name', F.concat(F.col('first_name'), F.lit(' '), F.col('last_name')))
    .toPandas()
    .head(3)
)

,player_id,team,first_name,last_name,position,batting_stance,at_bats,obp,full_name
0,1,Padres,Juan,Soto,RF,L,524.0,0.401,Juan Soto
1,2,Padres,Manny,Machado,3B,R,578.0,0.366,Manny Machado
2,3,Padres,Yu,Darvish,P,None,NaN,NaN,Yu Darvish


### Task 2: Apply a window function

#### Pandas solution

In [48]:
(
    df_pandas
    .head()
)

,player_id,team,first_name,last_name,position,batting_stance,at_bats,obp,full_name
0,1,Padres,Juan,Soto,RF,L,524.0,0.401,Juan Soto
1,2,Padres,Manny,Machado,3B,R,578.0,0.366,Manny Machado
2,3,Padres,Yu,Darvish,P,None,NaN,NaN,Yu Darvish
3,4,Padres,He-Seong,Kim,SS,R,517.0,0.325,He-Seong Kim
4,5,Padres,Jurickson,Profar,LF,S,575.0,0.331,Jurickson Profar


#### PySpark solution

In [54]:
(
    df_pyspark
    .filter(F.col('position')!='P')
    .withColumn('total_at_bats', F.sum('at_bats').over(W.Window.partitionBy('team')))
    .withColumn('pct_total_at_bats', F.col('at_bats')/F.col('total_at_bats'))
    .toPandas()
    .head(3)
)

,player_id,team,first_name,last_name,position,batting_stance,at_bats,obp,total_at_bats,pct_total_at_bats
0,1,Padres,Juan,Soto,RF,L,524.0,0.401,4649.0,0.112712
1,2,Padres,Manny,Machado,3B,R,578.0,0.366,4649.0,0.124328
2,4,Padres,He-Seong,Kim,SS,R,517.0,0.325,4649.0,0.111207
